<a href="https://www.kaggle.com/code/larsmagnusson/itf31519-feed-forward-networks?scriptVersionId=148591222" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heart-failure-prediction/heart.csv
/kaggle/input/life-expectancy-who/Life Expectancy Data.csv


In [2]:
# Load our dataset
dataset = pd.read_csv('/kaggle/input/heart-failure-prediction/heart.csv')
dataset.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [3]:
# Separate features from targets
features = dataset.drop('HeartDisease',axis=1)
targets = dataset['HeartDisease']

# Encode our targets using one-hot encoding
targets_onehot = pd.get_dummies(targets)
targets_onehot.head()

,0,1
0,1,0
1,0,1
2,1,0
3,0,1
4,1,0


In [4]:
targets.value_counts()

1    508
0    410
Name: HeartDisease, dtype: int64

In [5]:
features_onehot = pd.get_dummies(features)
features_onehot.shape

(918, 20)

In [6]:
features_onehot = features_onehot.astype('float32')
targets_onehot = targets_onehot.astype('float32')
features_onehot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Age                918 non-null    float32
 1   RestingBP          918 non-null    float32
 2   Cholesterol        918 non-null    float32
 3   FastingBS          918 non-null    float32
 4   MaxHR              918 non-null    float32
 5   Oldpeak            918 non-null    float32
 6   Sex_F              918 non-null    float32
 7   Sex_M              918 non-null    float32
 8   ChestPainType_ASY  918 non-null    float32
 9   ChestPainType_ATA  918 non-null    float32
 10  ChestPainType_NAP  918 non-null    float32
 11  ChestPainType_TA   918 non-null    float32
 12  RestingECG_LVH     918 non-null    float32
 13  RestingECG_Normal  918 non-null    float32
 14  RestingECG_ST      918 non-null    float32
 15  ExerciseAngina_N   918 non-null    float32
 16  ExerciseAngina_Y   918 non

In [7]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_targets, test_targets = train_test_split(features_onehot, targets_onehot, test_size=0.1, stratify=targets)

In [8]:
import tensorflow as tf

train_features_tensor = tf.convert_to_tensor(train_features)
train_targets_tensor = tf.convert_to_tensor(train_targets)
test_features_tensor = tf.convert_to_tensor(test_features)
test_targets_tensor = tf.convert_to_tensor(test_targets)

test_targets.shape

(92, 2)

In [9]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([Dense(20,activation='sigmoid'), Dense(12,activation='sigmoid'), Dense(2,activation="sigmoid")])
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [10]:
model.fit(train_features, train_targets, epochs=10, batch_size=24)

Epoch 1/10
35/35 [==============================] - 1s 2ms/step - loss: 0.8470 - accuracy: 0.5533
Epoch 2/10
35/35 [==============================] - 0s 2ms/step - loss: 0.6832 - accuracy: 0.5533
Epoch 3/10
35/35 [==============================] - 0s 2ms/step - loss: 0.6521 - accuracy: 0.5533
Epoch 4/10
35/35 [==============================] - 0s 2ms/step - loss: 0.6421 - accuracy: 0.5533
Epoch 5/10
35/35 [==============================] - 0s 2ms/step - loss: 0.6358 - accuracy: 0.6126
Epoch 6/10
35/35 [==============================] - 0s 2ms/step - loss: 0.6320 - accuracy: 0.6162
Epoch 7/10
35/35 [==============================] - 0s 2ms/step - loss: 0.6283 - accuracy: 0.6368
Epoch 8/10
35/35 [==============================] - 0s 2ms/step - loss: 0.6233 - accuracy: 0.7070
Epoch 9/10
35/35 [==============================] - 0s 2ms/step - loss: 0.6193 - accuracy: 0.6973
Epoch 10/10
35/35 [==============================] - 0s 2ms/step - loss: 0.6140 - accuracy: 0.7010


In [11]:
model.evaluate(test_features, test_targets)

3/3 [==============================] - 0s 4ms/step - loss: 0.5791 - accuracy: 0.7609


[0.5790501832962036, 0.760869562625885]

In [12]:
# Load and Pre-process the dataset. This code has been copied from the workshop on Regression for Life expectancy

# Load the dataset
dataset2 = pd.read_csv('/kaggle/input/life-expectancy-who/Life Expectancy Data.csv')
dataset2.head()

# Drop all rows where we are missing the life expectancy. Had to put the key in a list 
# for this to work in this notebook!?
dataset2 = dataset2.dropna(subset=['Life expectancy '])

# Fill the remaining missing values with the means for each column
float_cols_with_nas = ['Alcohol','Hepatitis B',' BMI ','Polio','Total expenditure','Diphtheria ','GDP','Population',' thinness  1-19 years',' thinness 5-9 years','Income composition of resources','Schooling']
dataset2[float_cols_with_nas] = dataset2[float_cols_with_nas].fillna(dataset2[float_cols_with_nas].mean())

# Convert the categorical columns to appropriate types
dataset2[['Country', 'Status']] = dataset2[['Country', 'Status']].astype('category')

# Encode the categorical columns using one-hot encoding
dataset2 = pd.get_dummies(dataset2)

# Separate into features/targets
targets2 = dataset2['Life expectancy ']
features2 = dataset2.drop('Life expectancy ', axis=1)

from sklearn.model_selection import train_test_split
# Split the data into 75/25 (as specified in the assignment)
train_targets2, test_targets2, train_features2, test_features2 = train_test_split(targets2,features2)

features2.head()

,Year,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,...,Country_Uruguay,Country_Uzbekistan,Country_Vanuatu,Country_Venezuela (Bolivarian Republic of),Country_Viet Nam,Country_Yemen,Country_Zambia,Country_Zimbabwe,Status_Developed,Status_Developing
0,2015,263.0,62,0.01,71.279624,65.0,1154,19.1,83,6.0,...,0,0,0,0,0,0,0,0,0,1
1,2014,271.0,64,0.01,73.523582,62.0,492,18.6,86,58.0,...,0,0,0,0,0,0,0,0,0,1
2,2013,268.0,66,0.01,73.219243,64.0,430,18.1,89,62.0,...,0,0,0,0,0,0,0,0,0,1
3,2012,272.0,69,0.01,78.184215,67.0,2787,17.6,93,67.0,...,0,0,0,0,0,0,0,0,0,1
4,2011,275.0,71,0.01,7.097109,68.0,3013,17.2,97,68.0,...,0,0,0,0,0,0,0,0,0,1


In [13]:
model2 = Sequential([Dense(units=204,activation='sigmoid'), Dense(units=1,activation="linear")])
model2.compile(optimizer='adam',loss='mean_squared_error',metrics=['mean_absolute_error'])
model2.fit(train_features2, train_targets2, epochs=20, batch_size=1)

Epoch 1/20
2196/2196 [==============================] - 6s 3ms/step - loss: 726.5529 - mean_absolute_error: 19.4996
Epoch 2/20
2196/2196 [==============================] - 6s 3ms/step - loss: 95.1260 - mean_absolute_error: 7.9895
Epoch 3/20
2196/2196 [==============================] - 6s 3ms/step - loss: 92.5338 - mean_absolute_error: 7.8619
Epoch 4/20
2196/2196 [==============================] - 6s 3ms/step - loss: 92.0531 - mean_absolute_error: 7.8769
Epoch 5/20
2196/2196 [==============================] - 6s 3ms/step - loss: 92.0992 - mean_absolute_error: 7.8715
Epoch 6/20
2196/2196 [==============================] - 6s 3ms/step - loss: 91.2767 - mean_absolute_error: 7.8688
Epoch 7/20
2196/2196 [==============================] - 6s 3ms/step - loss: 90.7330 - mean_absolute_error: 7.7931
Epoch 8/20
2196/2196 [==============================] - 6s 3ms/step - loss: 91.2431 - mean_absolute_error: 7.8317
Epoch 9/20
2196/2196 [==============================] - 6s 3ms/step - loss: 92.1028 - 

In [14]:
model2.evaluate(test_features2, test_targets2)
model2(test_features2[:20].to_numpy())

23/23 [==============================] - 0s 3ms/step - loss: 80.5129 - mean_absolute_error: 7.1368


<tf.Tensor: shape=(20, 1), dtype=float32, numpy=
array([[69.14499],
       [72.86983],
       [69.14499],
       [69.14499],
       [69.14499],
       [69.14499],
       [69.14499],
       [69.14499],
       [69.14499],
       [69.14499],
       [69.14499],
       [69.14499],
       [69.14499],
       [69.14499],
       [69.14499],
       [69.14499],
       [69.14499],
       [69.14499],
       [76.36868],
       [69.14499]], dtype=float32)>

In [15]:
from tensorflow.keras.layers import Normalization

normalization = Normalization(axis=-1)
normalization.adapt(train_features2)
model3 = Sequential([normalization,Dense(units=40,activation='sigmoid'),Dense(units=1)])
model3.compile(optimizer='sgd',loss='mean_squared_error',metrics=['mean_absolute_error'])
model3.fit(train_features2, train_targets2, epochs=20, batch_size=24)


Epoch 1/20
92/92 [==============================] - 1s 2ms/step - loss: 159.1219 - mean_absolute_error: 5.6901
Epoch 2/20
92/92 [==============================] - 0s 2ms/step - loss: 6.0528 - mean_absolute_error: 1.5960
Epoch 3/20
92/92 [==============================] - 0s 2ms/step - loss: 4.8975 - mean_absolute_error: 1.4148
Epoch 4/20
92/92 [==============================] - 0s 2ms/step - loss: 4.3668 - mean_absolute_error: 1.3108
Epoch 5/20
92/92 [==============================] - 0s 3ms/step - loss: 3.8582 - mean_absolute_error: 1.2279
Epoch 6/20
92/92 [==============================] - 0s 3ms/step - loss: 3.5987 - mean_absolute_error: 1.2012
Epoch 7/20
92/92 [==============================] - 0s 3ms/step - loss: 3.3489 - mean_absolute_error: 1.1313
Epoch 8/20
92/92 [==============================] - 0s 3ms/step - loss: 3.2720 - mean_absolute_error: 1.1263
Epoch 9/20
92/92 [==============================] - 0s 3ms/step - loss: 3.0002 - mean_absolute_error: 1.0893
Epoch 10/20
92/92

In [16]:
model3.evaluate(test_features2, test_targets2)
model3(test_features2[:40])

23/23 [==============================] - 0s 2ms/step - loss: 3.4191 - mean_absolute_error: 1.1564


<tf.Tensor: shape=(40, 1), dtype=float32, numpy=
array([[83.056114],
       [75.81015 ],
       [71.95679 ],
       [57.916462],
       [76.79432 ],
       [62.227673],
       [52.54907 ],
       [56.42404 ],
       [73.23101 ],
       [71.28678 ],
       [60.102203],
       [73.27007 ],
       [71.8586  ],
       [82.71493 ],
       [82.52937 ],
       [83.751595],
       [62.688854],
       [74.168465],
       [79.61701 ],
       [75.52957 ],
       [83.75596 ],
       [55.732075],
       [68.92953 ],
       [83.58776 ],
       [64.11484 ],
       [82.430405],
       [74.0527  ],
       [66.146675],
       [72.95106 ],
       [68.78286 ],
       [72.71769 ],
       [63.699024],
       [69.1498  ],
       [57.76127 ],
       [63.03988 ],
       [82.76477 ],
       [67.51234 ],
       [65.2183  ],
       [73.14222 ],
       [57.1143  ]], dtype=float32)>